In [17]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [57]:
from sht31 import SHT31
from machine import I2C, Pin, Timer
from common import task
import time
import pinout

In [65]:
filename = '/user/temp_humidity.csv'

def measure_cb_builder(s: SHT31):
    def cb(timer):
        sensor_value = s.get_temp_humi()
        time_val = time.ticks_ms()
        
        print(f'Measure {sensor_value} at {time_val}')
        
        with open(filename, 'a') as f:
            f.write(f'{time_val}, {sensor_value[0]}, {sensor_value[1]}\n')
    
    return cb

@task('1: File RW')
def task_1_file_rw():
    #Write csv header
    with open(filename, 'w') as f:
        f.write('#Time, Temp, Humidity\n')
    
    i2c = I2C(0, scl=Pin(pinout.scl), sda=Pin(pinout.sda), freq=400000)
    th_sensor = SHT31(i2c)
    
    print('Start sensor polling...')
    tim = Timer(0)
    tim.init(period=500, mode=Timer.PERIODIC, callback=measure_cb_builder(th_sensor))
    
    
    time.sleep_ms(5000)
    tim.deinit()
    
    print()
    print('Sensor polling end, collected csv data:')
    print()
    
    with open(filename, 'r') as f:
        print(f.read())

task_1_file_rw()

**************************************************
Start Of Task #1: File RW
**************************************************
Start sensor polling...
Measure (23.28832, 53.31044) at 6995192
Measure (23.33105, 53.16396) at 6995693
Measure (23.33105, 53.10903) at 6996193
Measure (23.31503, 52.99459) at 6996693
Measure (23.31503, 52.87404) at 6997193
Measure (23.31503, 52.80995) at 6997693
Measure (23.31503, 52.73213) at 6998193
Measure (23.31503, 52.57802) at 6998693
Measure (23.30168, 52.42084) at 6999193

Sensor polling end, collected csv data:

#Time, Temp, Humidity
6995192, 23.28832, 53.31044
6995693, 23.33105, 53.16396
6996193, 23.33105, 53.10903
6996693, 23.31503, 52.99459
6997193, 23.31503, 52.87404
6997693, 23.31503, 52.80995
6998193, 23.31503, 52.73213
6998693, 23.31503, 52.57802
6999193, 23.30168, 52.42084

**************************************************
End Of Task #1: File RW
**************************************************
Measure (23.2723, 52.31708) at 6999731
